In [1]:
import sys
sys.path += [
    './lib/',
    './lib/field_mappers'
]

In [2]:
from pymongo import MongoClient
import spacy

from extract_biographies import extract_biographies
from generate_sample_collection import copy_sample
from extract_first_sentence import FirstSentenceExtractor
from vectorizer import Vectorizer
from query import Searcher

In [3]:
SPACY_MODEL = 'en_vectors_web_lg'

In [4]:
db = MongoClient().who
# clean up test db
db.bios.remove({})
db.sample_bios.remove({})

/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  after removing the cwd from sys.path.


{'n': 10, 'ok': 1.0}

In [5]:
extract_biographies(collection_name='bios', limit=100)

inserting batch 0


In [6]:
db.bios.count_documents({})

100

In [7]:
copy_sample('bios', 'sample_bios', 10)
db.sample_bios.count_documents({})

10

In [8]:
FirstSentenceExtractor().extract('sample_bios', new_field_name='first_sentence')

insertation count: 0


In [9]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence'])

In [10]:
next(db.sample_bios.find({}))['first_sentence']

'Albert Einstein  was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics .'

In [11]:
nlp = spacy.load(SPACY_MODEL)

In [12]:
Vectorizer(nlp).vectorize_text(
    collection_name='sample_bios',
    text_field_name='first_sentence',
    vector_field_name='fresh_new_vectorization'
)

insertation count: 0


In [13]:
next(db.sample_bios.find({}))['fresh_new_vectorization'][:9]

[0.032865554094314575,
 0.17108696699142456,
 -0.14145277440547943,
 -0.05955364927649498,
 -0.03597114607691765,
 0.03867463022470474,
 -0.025338320061564445,
 -0.07364924997091293,
 0.0813150405883789]

In [14]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence', 'fresh_new_vectorization'])

In [15]:
Searcher(
    nlp,
    collection_name='sample_bios',
    vector_field_name='fresh_new_vectorization'
).query('religious ruler', limit=10)

generating vector similarity maps...


['Aga Khan I',
 'Aga Khan III',
 'Antonio Agliardi',
 'Alexander of Pherae',
 'Alexander II of Epirus',
 'Albert Einstein',
 'Albert the Bear',
 'Amalaric',
 'Alan Garner',
 'A. A. Milne']